# 과제: 병 걸린 벼 분류    
* 병걸린 벼 또는 정상 벼 라벨링 된 이미지 데이터 이용

In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [13]:
train=pd.read_csv("/kaggle/input/paddy-disease-classification/train.csv")
train

In [14]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import random

plt.figure(figsize=(10,6))
custom_palette0 = sns.color_palette("Paired", 9)
sns.countplot(x='label', data=train, palette=custom_palette0)
plt.title("count by disease type")

# > Image check

In [15]:
import pathlib
data_dir = pathlib.Path("../input/paddy-disease-classification/train_images")

In [16]:
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE)

In [17]:
class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

# > Data Augmentation

In [18]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from tensorflow.keras import layers
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import Callback

In [22]:
IMAGE_SIZE = 256
SEED = 54
im_shape = (IMAGE_SIZE, IMAGE_SIZE)
data_path = "/kaggle/input/paddy-disease-classification"


In [23]:
data_gen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.1,
        rotation_range=5,
        shear_range=0.3,
        zoom_range=0.3,
        width_shift_range=0.05,
        height_shift_range=0.05,
        horizontal_flip=True,
        vertical_flip=True,
)

test_gen = ImageDataGenerator(
        rescale=1./255,
)

In [24]:
print("Training Data:")
train_generator = data_gen.flow_from_directory(data_path + "/train_images", target_size=im_shape, shuffle=True, seed=SEED,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")

print("Validation Data:")
validation_generator = data_gen.flow_from_directory(data_path + "/train_images", target_size=im_shape, shuffle=False, seed=SEED,
                                                      class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")

print("Test Data:")
# test_generator = test_gen.flow_from_directory(data_path + "/test_images", target_size=im_shape, shuffle=False, seed=SEED,
#                                                      class_mode='categorical', batch_size=BATCH_SIZE)

test_path = '../input/paddy-disease-classification/test_images'
test_generator = test_gen.flow_from_directory(directory=test_path,target_size=im_shape,batch_size=BATCH_SIZE,
    classes=['.'],shuffle=False,)

classes = list(train_generator.class_indices.keys())
num_classes  = len(classes)
print(train_generator.class_indices)
print(len(train_generator.class_indices))

In [25]:
classes = list(train_generator.class_indices.keys())
print('Classes: '+str(classes))
num_classes  = len(classes)

In [26]:
plt.figure(figsize=(30,30))
images = train_generator.next()*255
for i, (image, l) in enumerate(zip(images[0][:9],images[1][:9])):
    ax = plt.subplot(330 + 1 + i)
    ax.set_title( classes[np.argmax(l)] )
    plt.imshow(image)
plt.show()

# > Model

In [27]:
SEED = 54
BATCH_SIZE = 64
EPOCHS = 100
LEARNING_RATE = 0.0001

model = tf.keras.Sequential([
tf.keras.layers.InputLayer(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
tf.keras.layers.Conv2D(128, (3,3), activation='relu' ),
tf.keras.layers.MaxPooling2D(2, 2),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dense(256, activation='relu'),
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dense(64, activation='relu'),
tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [28]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=5,
                            verbose=0, 
                            mode='auto')

lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                            factor=0.5,
                            patience=5)

callbacks = [early_stopping_callback, lr_reducer]

In [29]:
opt = tf.keras.optimizers.RMSprop(learning_rate= LEARNING_RATE)
# opt = tf.keras.optimizers.Adam(learning_rate= LEARNING_RATE, amsgrad=True)
metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 
           tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5)]

In [30]:
model.summary()
tf.keras.utils.plot_model(model, to_file='model.png',expand_nested=True,show_shapes=True)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer = opt, metrics= metrics)
history = model.fit(train_generator,callbacks=callbacks,epochs=EPOCHS,validation_data=validation_generator,verbose=1,)

In [ ]:
filenames=test_generator.filenames

results=pd.DataFrame({"image_id":filenames,
                      "label":predictions})
results.image_id = results.image_id.str.replace('./', '')
results.to_csv("submission.csv",index=False)
results.head()

In [ ]:
results['label'].value_counts()